### ML genre classification on FMA with feature selection using correlation

* use the 'medium' dataset to work with more training examplars
* note medium also only contains tracks where all tagged genres roll up to the same root genre
* drop highly correlated features

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import scipy as sp
import IPython.display as ipd

import matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import (train_test_split, GridSearchCV, RandomizedSearchCV)
from sklearn.metrics import (classification_report, confusion_matrix, ConfusionMatrixDisplay, f1_score)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV

import utils

RANDOM_STATE = 53

In [2]:
(features, tracks) = utils.load_features()
features.shape, tracks.shape

((106574, 518), (106574, 52))

In [3]:
small = tracks[('set', 'subset')] == 'small'
medium = tracks[('set', 'subset')].isin(['small','medium'])
X = features[medium]
y = tracks[medium][('track','genre_top')]

print(X.shape, y.shape)

(25000, 518) (25000,)


#### arbitrarily eliminate bottom 3 genres due to lack of exemplars


In [4]:
prune = ~y.isin(['Spoken','Blues','Easy Listening'])
y = y[prune]
X = X[prune]

#### build training/test sets

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2,
                                                    random_state=RANDOM_STATE,
                                                    shuffle=True,
                                                    stratify=y)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### run correlation

In [16]:
print(X_train_scaled.shape)
correlations = np.corrcoef(X_train_scaled, rowvar=False)

keep_cols = np.full(X_train_scaled.shape[1], True)

for i in range(correlations.shape[0]):
    for j in range(i+1, correlations.shape[0]):
        val = correlations[i,j]
        if abs(val) > 0.90:
            keep_cols[i] = False
            

X_train_corr = X_train_scaled[:,keep_cols]
X_test_corr = X_test_scaled[:,keep_cols]
print(X_train_corr.shape, X_test_corr.shape)


(19829, 518)
(19829, 401) (4958, 401)


#### 4 Basic ML classifiers with classifiers with Correlation reduced features

In [17]:
classifiers = {'SVC': SVC(kernel='linear', random_state=RANDOM_STATE),
               'SVC-RBF': SVC(kernel='rbf', random_state=RANDOM_STATE),
               'LR' : LogisticRegression(random_state=RANDOM_STATE),
               'KNN' :KNeighborsClassifier()
              }

for (name, cl) in classifiers.items():
    cl.fit(X_train_corr, y_train)
    y_pred = cl.predict(X_test_corr)
    print(classification_report(y_test, y_pred))

                     precision    recall  f1-score   support

          Classical       0.75      0.84      0.79       124
            Country       0.50      0.06      0.10        35
         Electronic       0.65      0.82      0.73      1263
       Experimental       0.48      0.38      0.42       450
               Folk       0.65      0.63      0.64       304
            Hip-Hop       0.71      0.63      0.67       440
       Instrumental       0.56      0.46      0.50       270
      International       0.61      0.44      0.51       204
               Jazz       0.62      0.31      0.41        77
Old-Time / Historic       0.96      0.94      0.95       102
                Pop       0.43      0.06      0.11       237
               Rock       0.75      0.87      0.81      1421
           Soul-RnB       1.00      0.03      0.06        31

           accuracy                           0.68      4958
          macro avg       0.67      0.50      0.52      4958
       weighted avg   

C:\Users\orovi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\orovi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\orovi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                     precision    recall  f1-score   support

          Classical       0.72      0.78      0.75       124
            Country       0.00      0.00      0.00        35
         Electronic       0.62      0.85      0.71      1263
       Experimental       0.50      0.34      0.40       450
               Folk       0.62      0.58      0.60       304
            Hip-Hop       0.75      0.55      0.64       440
       Instrumental       0.54      0.38      0.44       270
      International       0.67      0.24      0.35       204
               Jazz       0.00      0.00      0.00        77
Old-Time / Historic       0.96      0.95      0.96       102
                Pop       0.00      0.00      0.00       237
               Rock       0.70      0.89      0.78      1421
           Soul-RnB       0.00      0.00      0.00        31

           accuracy                           0.66      4958
          macro avg       0.47      0.43      0.43      4958
       weighted avg   

C:\Users\orovi\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\orovi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\orovi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in l

                     precision    recall  f1-score   support

          Classical       0.67      0.72      0.70       124
            Country       0.00      0.00      0.00        35
         Electronic       0.64      0.80      0.71      1263
       Experimental       0.43      0.31      0.36       450
               Folk       0.58      0.57      0.57       304
            Hip-Hop       0.64      0.60      0.62       440
       Instrumental       0.49      0.39      0.43       270
      International       0.46      0.28      0.35       204
               Jazz       0.82      0.12      0.20        77
Old-Time / Historic       0.92      0.94      0.93       102
                Pop       0.29      0.04      0.07       237
               Rock       0.71      0.87      0.78      1421
           Soul-RnB       0.00      0.00      0.00        31

           accuracy                           0.64      4958
          macro avg       0.51      0.43      0.44      4958
       weighted avg   

In [18]:
classifiers = {'SVC': SVC(kernel='linear', class_weight='balanced', random_state=RANDOM_STATE),
               'SVC-RBF': SVC(kernel='rbf', class_weight='balanced',random_state=RANDOM_STATE),
               'LR' : LogisticRegression(class_weight='balanced', max_iter=5000, random_state=RANDOM_STATE),
              }

for (name, cl) in classifiers.items():
    cl.fit(X_train_corr, y_train)
    y_pred = cl.predict(X_test_corr)
    print(classification_report(y_test, y_pred))

                     precision    recall  f1-score   support

          Classical       0.64      0.86      0.74       124
            Country       0.16      0.63      0.25        35
         Electronic       0.77      0.57      0.66      1263
       Experimental       0.42      0.43      0.42       450
               Folk       0.61      0.66      0.63       304
            Hip-Hop       0.58      0.72      0.65       440
       Instrumental       0.40      0.58      0.47       270
      International       0.43      0.52      0.47       204
               Jazz       0.33      0.61      0.43        77
Old-Time / Historic       0.94      0.94      0.94       102
                Pop       0.20      0.27      0.23       237
               Rock       0.90      0.68      0.77      1421
           Soul-RnB       0.18      0.55      0.28        31

           accuracy                           0.61      4958
          macro avg       0.50      0.62      0.53      4958
       weighted avg   